In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Data

In [125]:
train_df = pd.read_json('train.json',  lines = True).drop(['index'], axis = 1)
test_df = pd.read_json('test.json', lines = True).drop(['index'], axis = 1)

def cut_short(seq, maxlen = 107):
    return seq[:maxlen]

    
test_df[['sequence']]=test_df['sequence'].apply(cut_short)
test_df[['structure']]=test_df['structure'].apply(cut_short)
test_df[['predicted_loop_type']]=test_df['predicted_loop_type'].apply(cut_short)


Index(['id', 'sequence', 'structure', 'predicted_loop_type', 'seq_length',
       'seq_scored'],
      dtype='object')
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
1

107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107
107


array([[[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 3,  2,  8],
        [ 3,  2,  8],
        [ 4,  2,  8]],

       [[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 4,  1, 12],
        [ 3,  1, 12],
        [ 3,  1, 12]],

       [[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 3,  2,  8],
        [ 3,  2,  8],
        [ 4,  2,  8]],

       ...,

       [[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 5,  1, 12],
        [ 5,  1, 12],
        [ 6,  1, 12]],

       [[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 4,  1, 12],
        [ 4,  1, 12],
        [ 4,  1, 12]],

       [[ 5,  2,  8],
        [ 5,  2,  8],
        [ 3,  2,  8],
        ...,
        [ 4,  1, 12],
        [ 6,  1, 12],
        [ 4,  1, 12]]])

In [3]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSXZ')}

target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )

train_inputs = preprocess_inputs(train_df.loc[train_df.SN_filter == 1])
train_labels = np.array(train_df.loc[train_df.SN_filter == 1][target_cols].values.tolist()).transpose((0, 2, 1))

In [4]:
from sklearn.model_selection import train_test_split

seed = 123
X_train, X_test, Y_train, Y_test = train_test_split(train_inputs, train_labels, test_size = 0.1, random_state = seed)

print(np.shape(X_train)) #1430 samples de séquences de 107 de long et de 3=sequence,structure, predicted loop type

(1430, 107, 3)


# Model

In [6]:
import tensorflow.keras.layers as L
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True))


def build_model(seq_len=107, pred_len=68, dropout=0.5, embed_dim=75, hidden_dim=128):
    inputs = L.Input(shape=(seq_len, 3))

    embed = L.Embedding(input_dim=len(token2int), output_dim=embed_dim)(inputs)
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))

    hidden = gru_layer(hidden_dim, dropout)(reshaped)
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    
    # Since we are only making predictions on the first part of each sequence, we have
    # to truncate it
    truncated = hidden[:, :pred_len]
    
    out = L.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=inputs, outputs=out)

    model.compile(tf.keras.optimizers.Adam(), loss='mse',  metrics=['accuracy'])
    
    return model

In [25]:
def prepare_window_training(base_index, n_bases,X_train, Y_train, X_test, Y_test):
   
    start_index=int(i-(window_size-1)/2)
    stop_index=int(i+(window_size-1)/2 +1) #not included
    full_array=[]
    for sample_index in range(len(X_train)):
        sample_array=X_train[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array.append(studied_array)
    
    full_array_test=[]
    for sample_index in range(len(X_test)):
        sample_array=X_test[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array_test.append(studied_array)

    X_train_window=np.array(full_array)
    Y_train_window=Y_train[:,i,:]
    X_test_window=np.array(full_array_test)
    Y_test_window=Y_test[:,i,:]
    
    return X_train_window, Y_train_window, X_test_window, Y_test_window

# Window mode


In [135]:
def prepare_window_prediction(base_index, n_bases, X_to_predict):
   
    start_index=int(i-(window_size-1)/2)
    stop_index=int(i+(window_size-1)/2 +1) #not included
    full_array=[]
    for sample_index in range(len(X_to_predict)):
        sample_array=X_to_predict[sample_index]

        if start_index<0:
            out_array=np.array([[14]*len(sample_array[0])]*(-int(start_index)))
            studied_cut_array=sample_array[0:stop_index]
            studied_array=np.concatenate((out_array,studied_cut_array))

        elif stop_index>68:
            out_array=np.array([[14]*len(sample_array[0])]*(int(stop_index-68)))
            studied_cut_array=sample_array[start_index:68]
            studied_array=np.concatenate((studied_cut_array,out_array))

        else :
            studied_array=sample_array[start_index:stop_index]
        full_array.append(studied_array)
        
    return np.array(full_array)

In [ ]:
#mode fenêtré:
window_size=21 #doit être un nombre impair
X_train=X_train[:,:68,:]
X_test=X_test[:,:68,:]

X_to_predict = preprocess_inputs(test_df)[:,:68,:]
Y_predicted = np.empty(shape=(len(X_to_predict), 68, 5))
print(len(X_to_predict))
for i in range(68): #on parcourt toute la séquence
    
    print('Training for base', i, '===================================')

    X_train_window, Y_train_window, X_test_window, Y_test_window = prepare_window_training(i, 68, 
                                                                                   X_train, Y_train, 
                                                                                   X_test, Y_test)
    #ici il faut faire le fit sur le full_array de dimensions 2160*21*14=nombredeséquences*window_size*nombredeparamètresd'input
    #ensuite on lance le modèle sur les vraies données, 
    #et on stocke le résultat [a,b,c] dans l'item i une liste du type [[1, 0, 0], [0, 1, 0.3], [0.4, 0.7, 1], ...], de dimensions 68*3
    
    model = build_model(seq_len=21, embed_dim=len(token2int))
    model.summary()
    Y_train_window=np.array([np.tile(Y_train_window[i],(21,1)) for i in range(len(Y_train_window))])
    Y_test_window=np.array([np.tile(Y_test_window[i],(21,1)) for i in range(len(Y_test_window))])

    print(np.shape(X_train_window))
    print(np.shape(Y_train_window))
    print(np.shape(X_test_window))
    print(np.shape(Y_test_window))
    
    history = model.fit(
    X_train_window, 
    Y_train_window,
    validation_data=(X_test_window, Y_test_window),
    batch_size=32,
    epochs=30,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(patience=5),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
    )
    plt.subplot()
    plt.plot(history.history['val_loss'])
    plt.plot(history.history['loss'])
    plt.legend(['val_loss', 'loss'], loc='upper right')

    
    print('Evaluation for base', i, '===================================')

# la prédiction qui suit ne marche pas encore 
    X_predic=prepare_window_prediction(i,68,X_to_predict)
    
    for seq in range(len(X_predic)) :
        if seq%100==0:
            print(seq)
        base_predict=model.predict(X_predic[seq])
        mean_predict=np.mean(base_predict[0], axis=0)
        Y_predicted[seq, i, :]=mean_predict
#         print(mean_predict)

3634
Training for base 0 ===================================
Model: "functional_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(None, 21, 3)]           0         
_________________________________________________________________
embedding_37 (Embedding)     (None, 21, 3, 15)         225       
_________________________________________________________________
tf_op_layer_Reshape_37 (Tens [(None, 21, 45)]          0         
_________________________________________________________________
bidirectional_111 (Bidirecti (None, 21, 256)           134400    
_________________________________________________________________
bidirectional_112 (Bidirecti (None, 21, 256)           296448    
_________________________________________________________________
bidirectional_113 (Bidirecti (None, 21, 256)           296448    
__________________________________________________________

Dans la préparation des données, (mode fenêtré), au lieu de mettre des -1 (ou des 14 en fait) à la fin de la séquence on peut mettre les vraies bases et structures

In [141]:
i=0
X_to_predict = preprocess_inputs(test_df)
X_predic=prepare_window_prediction(i,68,X_to_predict)
print(np.shape(X_predic))

seq=0

Y_predicted = np.empty(shape=(len(X_to_predict), 68, 5))

base_predict=model.predict(X_predic[seq])
Y_predicted[seq, i, :]=np.mean(base_predict[0], axis=0)



(3634, 21, 3)
[[[0.41674286 0.4671347  1.4985886  0.358269   0.4203465 ]
  [0.5142655  0.5757119  1.7937676  0.42679492 0.52134496]
  [0.57527834 0.63552916 1.9683814  0.46916822 0.5774817 ]
  [0.61128104 0.66487485 2.0638163  0.49268258 0.6061621 ]
  [0.62924284 0.6739719  2.1037922  0.5021365  0.61695665]
  [0.6331947  0.6684167  2.1020367  0.5005853  0.6151986 ]
  [0.62550944 0.65136707 2.0671666  0.4901074  0.6039386 ]
  [0.6079748  0.6251675  2.0066411  0.47261629 0.5852851 ]
  [0.58291554 0.5928929  1.9309244  0.45083216 0.561608  ]
  [0.5546226  0.5601406  1.8583722  0.42946038 0.536874  ]
  [0.53285545 0.54467964 1.8322926  0.4198663  0.52361745]
  [0.532381   0.557995   1.874204   0.42782936 0.533457  ]
  [0.542587   0.5824399  1.9385228  0.4423809  0.5521658 ]
  [0.5614021  0.61198395 2.0147612  0.46297425 0.57389116]
  [0.5848469  0.64236414 2.0905437  0.4865484  0.5957621 ]
  [0.6097378  0.66968197 2.154523   0.5096239  0.61462426]
  [0.6334111  0.68932366 2.1914537  0.5274

In [ ]:
Y_predicted